In [2]:
import os
import numpy as np
import random
import pickle
import copy
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import KFold

np.random.seed(0)
random.seed(0)

In [ ]:
new_num_kfolds = 7

In [32]:
# INTRA

cv_results_path = r"C:\Users\kdmen\Desktop\Research\personalization-privacy-risk\PythonVersion\PythonSimsRevamp\results\FinalSims"
nofl_filename = "FULLSCIPYMIN_NOFL_KFold0.h5" 

directory_str = "09-19_22-56_NOFL"
h5_path = os.path.join(cv_results_path, directory_str, nofl_filename)
with h5py.File(h5_path, 'r') as f:
    a_group_key = list(f.keys())#[0]
    print(a_group_key)

    local_test_error_log = f['local_test_error_log'][()]
    local_train_error_log = f['local_train_error_log'][()]
    
    client_local_model_log_keys = list(f['client_local_model_log'].keys())
    #print("Keys in 'client_local_test_log':", client_local_model_log_keys)
    client_local_model_log_data = {}
    for key in client_local_test_log_keys:
        try:
            client_local_model_log_data[key] = f['client_local_model_log'][key][:]
        except KeyError:
            pass
            
    client_local_test_log_keys = list(f['client_local_test_log'].keys())
    # Iterate over the keys and extract the corresponding data
    client_local_test_log_data = {}
    for key in client_local_test_log_keys:
        client_local_test_log_data[key] = f['client_local_test_log'][key][:]

    client_gradnorm_log_keys = list(f['gradient_norm_lists_by_client'].keys())
    # Iterate over the keys and extract the corresponding data
    client_gradnorm_log_data = {}
    for key in client_gradnorm_log_keys:
        client_gradnorm_log_data[key] = f['gradient_norm_lists_by_client'][key][:]


['client_local_model_log', 'client_local_test_log', 'gradient_norm_lists_by_client', 'local_test_error_log', 'local_train_error_log']
